In [1]:
from collections import defaultdict
import numpy as np

In [104]:
lines = [s.strip() for s in list(open("rosalind_ba3j.txt", "r"))]
k, d = [int(val) for val in lines[0].split()]
paired_reads = [s.split('|') for s in lines[1:]]

graph = defaultdict(list)
nodes = []
source = None
balance = defaultdict(int)
for l, r in paired_reads:
    pref = (l[:-1], r[:-1])
    suff = (l[1:], r[1:])
    nodes.extend([pref, suff])
    graph[pref].append(suff)
    balance[pref] -= 1
    balance[suff] += 1

balance = np.array([balance[v] for v in nodes])
source = nodes[np.where(balance < 0)[0][0]]

curr_path, ecycle = [source], []
curr_node = source
while curr_path:
    if len(graph[curr_node]):
        curr_path.append(curr_node)
        curr_node = graph[curr_node].pop()
    else:
        ecycle.append(curr_node)
        curr_node = curr_path.pop()
ecycle.reverse()
ecycle = np.vstack(ecycle)

# suffix(read1 from node i) = prefix(read1 from node i+1)
# ==> the prefix of read1 from node i+1 is already in read1 from node i
# ==> we join last letters from every read1
# *the same for the read2.
prefs = ecycle[0, 0] + ''.join([s[k-2:] for s in ecycle[1:, 0]])
suffs = ecycle[0, 1] + ''.join([s[k-2:] for s in ecycle[1:, 1]])
answer = prefs[:d + k] + suffs
print(answer)

TTTTGCACCGTAAATGAAGGGGCGCTCGATAGCCAATTTAACTTGACGCGAATCTCATGATTGACCCCTTCCTTTGACAATACCCCTTGGATGCCAATCGAGTCCTGGGAAGGATCCCCATTATGAGGCGAATACGGGGGGACCGAATTCTATGCAGAGAGAATTTCAGCTGGCGTCCGGCTGGAGCGATATTAGGTTTGCGGACGTCGTCTAGAATTATGGGCCAGTGTCCGGGTACTGGCTTTCGCGAGCAATCCTCTAGTCCTTACGCGAGATGAATATATTCTGGATCTTAGAAGTTATGGGTCATCCGAAATACCCTCGATTGTAGTCGGGTCGGAGCTTGATCATGCCTATTAGCACTTCATCCTCGTCCCAGTTCGATGAAGAAAGGTTCGTAATATTTCCCCGATATGGATGCCAATCGAGTCCTGGGAAGGATCCGTGCTTTTAATTCGTTTGCCAGGGGACTATGTGATACTTGGATGCCAATCGAGTCCTGGGAAGGATCCCCAATATGTGACGTGCGCAGTATTCGAGGAATCTCTTCTGGATGCCAATCGAGTCCTGGGAAGGATCCGTAATTCGCATGCTCCGAGCCGTTGCCGCAGGCTTGGATGCCAATCGAGTCCTGGTGGATGCCAATCGAGTCCTGGGAAGGATCCGTTGGATGCCAATCGAGTCCTGGGAAGGATCCGGATGCCAATCGAGTCCTGGGAAGGATCCAAGGATCCCCTCAGCTGCTACCGCCGAATCCATCGGCCGTGGCTCCGCTAAAGCAGCTCCGGATGCTTGTTGCTCAGTGTTCCCTCTTCATGTCGGGTCCTGGACACCCAACCGATTTGAGTTCGCAGTGATGGATGCCAATCGAGTCCTGGGAAGGATCCTCCAGAGCGGTCGCCCGTAATGGCTGACCTTCTAGAAAGAGAATTGGTGTCAAGCTTGGAAGGCTTGCTCAAAGGCTTGTCGATCAGGATATGGATGCCAATCGAGTCCTGGG

In [23]:
prefs2 = ecycle[0, 0] + ''.join([s[-1] for s in ecycle[1:, 0]])
suffs2 = ecycle[0, 1] + ''.join([s[-1] for s in ecycle[1:, 1]])
answer2 = prefs2[:d + k] + suffs2

In [24]:
answer2

'GTGGTCGTGAGATGTTGA'

In [72]:
ecycle

array([['GTG', 'GTG'],
       ['TGG', 'TGA'],
       ['GGT', 'GAG'],
       ['GTC', 'AGA'],
       ['TCG', 'GAT'],
       ['CGT', 'ATG'],
       ['GTG', 'TGT'],
       ['TGA', 'GTT'],
       ['GAG', 'TTG'],
       ['AGA', 'TGA']], dtype='<U3')

In [73]:
reads1, reads2 = zip(*ecycle)

In [33]:
part1 = reads1[0][:-1] + ''.join([p[-1] for p in reads1])

In [34]:
part2 = reads2[0][:-1] + ''.join([p[-1] for p in reads2])

In [36]:
(part1[:d + k] + part2) == 'GTGGTCGTGAGATGTTGA'

True

In [85]:
def build_path_graph(Adj, paired_reads):
    s = None
    vertices = []
    # degree_balance[v] = in_degree[v] - out_degree[v]
    degree_balance = defaultdict(int)
    for read_1, read_2 in paired_reads:
        prefix = (read_1[:-1], read_2[:-1])
        suffix = (read_1[1:], read_2[1:])
        vertices.extend([prefix, suffix])
        Adj[prefix].append(suffix)
        degree_balance[prefix] -= 1
        degree_balance[suffix] += 1
    degree_balance = np.array([degree_balance[v] for v in vertices])

    # for source in_degree[v] = 0 and out_degree[v] > 0
    # ==> degree_balance[s] < 0
    s = vertices[np.where(degree_balance < 0)[0][0]]
    return s

In [86]:
def dfs(s, Adj):
    euler = []
    path = [s]
    u = s
    while path:
        if len(Adj[u]):
            path.append(u)
            u = Adj[u].pop()
        else:
            euler.append(u)
            u = path.pop()
    euler.reverse()
    euler = np.vstack(euler)
    
    return euler

In [87]:
def get_string(euler, k, d):
    reads1, reads2 = zip(*euler)
    part1 = reads1[0][:-1] + ''.join([p[-1] for p in reads1])
    part2 = reads2[0][:-1] + ''.join([p[-1] for p in reads2])

    return part1[:k+d] + part2

In [96]:
def string_from_read_pairs(k, d, paired_reads):
    
    Adj = defaultdict(list)
    s = build_path_graph(Adj, paired_reads)
    euler = dfs(s, Adj)
    
    return get_string(euler, k, d)

In [109]:
def main():
    
    file = open('rosalind_ba3j.txt', 'r')
    
    k, d = list(map(int, next(file).split()))
    
    paired_reads = []
    for s in file:
        s = s.strip()
        paired_reads.append(s.split('|'))
    
    print(string_from_read_pairs(k, d, paired_reads))

    file.close()

In [108]:
if __name__ == '__main__':
    main()

GTGGTCGTGAGATGTTGA


In [99]:
'GTGGTCGTGAGATGTTGA' == 'GTGGTCGTGAGATGTTGA'

True